In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


Load Dataset

In [ ]:
df= pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')
df.head()

In [ ]:
df.groupby('area_type')['area_type'].agg('count')

In [ ]:
df1=df.drop(['area_type','availability','society','balcony'],axis= 1)
df1.head(5)

In [ ]:
df1.isnull().sum()

In [ ]:
df2=df1.dropna()
df2.isnull().sum()

In [ ]:
df2['size'].unique()


In [ ]:
df2['BHK']= df2['size'].apply(lambda x: int(x.split(' ')[0]))


In [ ]:
df2.head()

In [ ]:
df3=df2.drop(['size'],axis=1)
df3.head()

In [ ]:
df3['total_sqft'].unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df3[~df3['total_sqft'].apply(is_float)].head()


In [ ]:
def num(x):
    tokens=x.split('-')
    if len(tokens)==2:
        return(float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None
            
    

In [ ]:
df4=df3.copy()
df4['total_sqft'] =df4['total_sqft'].apply(num)
df4.head(10)


In [ ]:
df4.loc[9]

Finally I removed the null values and changed the unstructured values in size,total_sqft columns to structured one.

Feature Engineering

In [ ]:
df5=df4.copy()
df5['Price_per_sqft']=df5['price']*1000000/df5['total_sqft']
df5.head()

In [ ]:
len(df5['location'].unique())

In [ ]:
df5['location']= df5['location'].apply(lambda x: x.strip())
location_stats = df5.groupby('location')['location'].agg('count').sort_values(ascending = False)
location_stats

In [ ]:
len(location_stats[location_stats<=10])

In [ ]:
location_less= location_stats[location_stats<=10]
location_less

In [ ]:
df5['location']=df5.location.apply(lambda x : 'Other' if x in location_less else x)
len(df5['location'].unique())

In [ ]:
df5.head(10)

Finally I convert the total square feet along with price to price per square feet and also changed minimum located area to one single category 'other'.

**OUTLIER REMOVAL********

In [ ]:
df5[df5.total_sqft/df5.BHK<300].head()

In [ ]:
df5.shape

In [ ]:
df6= df5[~(df5.total_sqft/df5.BHK<300)]
df6.shape

In [ ]:
df6.Price_per_sqft.describe()

In [ ]:
def remove(df):
    df_out= pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.Price_per_sqft)
        st=np.std(subdf.Price_per_sqft)
        redu=subdf[(subdf.Price_per_sqft>(m-st))& (subdf.Price_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,redu],ignore_index=True)
    return df_out
df7= remove(df6)
df7.shape
    

In [ ]:
def scatter_plot(df,location):
    BHK2= df[(df.location == location)& (df.BHK == 2)]
    BHK3= df[(df.location == location)& (df.BHK == 3)]
    #matplotlib.rcParams['figure','figsize']=(15,10)
    plt.scatter(BHK2.total_sqft,BHK2.price,color='blue',label='2BHK',s=50)
    plt.scatter(BHK3.total_sqft,BHK3.price,color='red',marker ='+',label='3BHK',s=70)
    plt.xlabel('Total Square Feet')
    plt.ylabel('price')
    plt.title(location)
    plt.legend()
scatter_plot(df7,'Rajaji Nagar')

In [ ]:
def remove_outlier(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        BHK_stats = {}
        for BHK, BHK_df in location_df.groupby('BHK'):
            BHK_stats[BHK] = {
                'mean': np.mean(BHK_df.Price_per_sqft),
                'std': np.std(BHK_df.Price_per_sqft),
                'count': BHK_df.shape[0]
            }
        for BHK, BHK_df in location_df.groupby('BHK'):
            stats = BHK_stats.get(BHK-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, BHK_df[BHK_df.Price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_outlier(df7)
# df8 = df7.copy()
df8.shape    

In [ ]:
scatter_plot(df8,'Rajaji Nagar')

In [ ]:
plt.hist(df8.Price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df8.bath.unique()

In [ ]:
df8[df8.bath>10]

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Bathroom")
plt.ylabel("Count")

In [ ]:
df8[df8.bath>df8.BHK+2]

In [ ]:
df9 =df8[df8.bath<df8.BHK+2]
df9.shape